In [26]:
from bs4 import BeautifulSoup
import requests
from itertools import chain
import pandas as pd
import re
import numpy as np

In [42]:
dfz = pd.read_excel('cities.xlsx')

In [49]:
x = list(set(dfz['city'].dropna().tolist() + dfz['city2'].tolist()))

In [28]:
x = list(set(dfz['city'].tolist() + dfz['city2'].tolist()))

In [39]:
x = dfz['city2'].tolist()

In [51]:
def create_frame(ls):
    
    total = pd.DataFrame()
    
    for city in ls:
        new =  []

        page = requests.get("https://www.numbeo.com/cost-of-living/in/" + city)
        soup = BeautifulSoup(page.content)
        data = soup.find_all("table",{"class":"data_wide_table new_bar_table"})
        data_cost = soup.find_all("span",{"class":"emp_number"})
        try:
            cost_index = re.search('>(.*)\<', str(data_cost[-1])).group(1)
        except IndexError:
            cost_index = '0'
        
        l = [item.text.split('€') for item in data]
        newlist = list(chain(*l))

        new.append('City' + '  '  + city)
        new.append('Cost_index' + '  ' + cost_index)

        for i in range(len(newlist)):
            try:
                new.append(re.search('\n(.*)\xa0', newlist[i]).group(1))
            except AttributeError as e:
                pass
            
        if len(new) != 56:
            pass
        
        else:
            for i in range(len(new)):
                total = total.append(pd.DataFrame({'':new[i].split('  ')[0], 'Cost':new[i].split('  ')[1]},index=[0]), ignore_index=True)
            
            headers = total.T.iloc[0]
            headers = pd.DataFrame(headers[:56])
            
    values = []
    n = len(total)/56
    df_list = np.vsplit(total, n)

    for i in df_list:
        values.append(i['Cost'].tolist())
    
    v = pd.DataFrame(values)
    zz = pd.DataFrame(headers).T
    xx = pd.concat([zz,v])
    new_header = xx.iloc[0]
    xx = xx[1:]
    xx.columns = new_header
    
    final = xx.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    final = final.drop(['Water (0.33 liter bottle)'], axis = 1)
    
    zz = final['City']
    z5 = final.loc[:, final.columns != 'City']
    z5 = z5.replace(',', '', regex=True)
    z39 = z5.apply(pd.to_numeric, errors='coerce')
    df = pd.concat([zz, z39],axis=1)
    
            
    return df

In [52]:
df = create_frame(x)

In [53]:
df

,City,Cost_index,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Beer (0.5 liter draught),Imported Beer (0.33 liter bottle),Cappuccino (regular),Coke/Pepsi (0.33 liter bottle),"Milk (regular), (1 liter)",...,"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax)
0,Eindhoven,73.02,15.00,60.0,8.00,5.00,3.5,2.79,2.62,1.00,...,32.21,84.65,117.87,1122.50,862.19,1556.86,1255.17,3617.22,2684.38,2595.77
1,Tampere,72.83,12.00,70.0,8.00,6.00,6.0,3.42,2.43,0.94,...,37.67,84.92,133.22,721.05,562.05,1188.08,884.38,4598.33,2969.30,2326.64
2,Rotterdam,69.10,14.00,50.0,8.00,4.00,4.0,2.90,2.24,0.91,...,31.80,89.79,138.16,1145.17,889.00,1963.64,1302.27,3777.29,2640.95,2334.60
3,Nuremberg,66.08,10.00,49.0,8.00,3.40,3.4,2.93,2.56,0.80,...,33.57,81.89,107.00,721.90,542.11,1165.17,938.33,4644.44,3637.50,2280.56
4,Kosice,43.65,5.70,27.0,5.10,1.50,1.8,1.74,1.34,0.63,...,27.49,56.88,66.66,480.00,372.22,715.62,537.50,1905.56,1463.33,806.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Groningen,73.09,15.00,60.0,8.00,4.25,3.5,3.13,2.32,0.86,...,35.00,96.12,141.00,718.00,627.08,1262.50,1016.67,2920.00,2475.00,2150.00
92,Marbella,0.00,8.81,30.0,7.00,2.20,3.0,1.45,1.34,0.71,...,24.44,54.09,55.00,768.18,644.44,1487.50,1085.71,3625.00,2760.00,1380.00
93,Athens,56.40,10.00,40.0,6.15,4.00,4.5,3.21,1.69,1.31,...,28.88,72.96,92.44,407.57,352.27,731.88,655.25,2054.18,1938.06,779.65
94,Bonn,66.01,9.50,55.0,7.50,3.50,3.5,2.61,2.06,0.74,...,35.50,72.88,100.43,784.62,586.15,1507.69,1138.46,7433.33,4933.33,2431.83


In [56]:
df.loc[df['Average Monthly Net Salary (After Tax)'] == df['Average Monthly Net Salary (After Tax)'].max(), 'City']

27    Luxembourg
Name: City, dtype: object

In [57]:
df.loc[df['Average Monthly Net Salary (After Tax)'] == df['Average Monthly Net Salary (After Tax)'].min(), 'City']

74    Podgorica
Name: City, dtype: object